## Apply enrichment method

This notebook plugs in other gene set enrichment methods to demonstrate that our method, SOPHIE, can be inserted into different pipelines and work with other methods

In [1]:
%load_ext autoreload
%load_ext rpy2.ipython
%autoreload 2

import os
import sys
import pandas as pd
import numpy as np
import pickle

from rpy2.robjects import pandas2ri
pandas2ri.activate()

from ponyo import utils
from generic_expression_patterns_modules import ranking

np.random.seed(123)

/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
# Read in config variables
base_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))

config_filename = os.path.abspath(
    os.path.join(base_dir, "configs", "config_human_general.tsv")
)

params = utils.read_config(config_filename)

In [3]:
# Load params
local_dir = params["local_dir"]
project_id = params['project_id']
statistic = params['gsea_statistic']
hallmark_DB_filename = params["pathway_DB_filename"]
num_runs = params["num_simulated"]
dataset_name = params['dataset_name']

# Enrichment method
# enrichment_method = ["GSEA", GSVA", "ROAST", "CAMERA", "OSA"]
# If enrichment_method == "GSEA" then use "padj" to rank
# If enrichment_method == "GSVA" then use "ES" to rank
# If enrichment_method == "ROAST" or "CAMERA" then use "FDR" to rank
# If using "OSA" then use
enrichment_method = "CAMERA"
col_to_rank_pathways = "FDR"

In [4]:
# Load DE stats directory
DE_stats_dir = os.path.join(local_dir, "DE_stats")

# Template experiment gene expression
template_expression_filename = os.path.join(base_dir, dataset_name, params["processed_template_filename"])

# Template experiment DE stats
template_DE_stats_filename = os.path.join(
    DE_stats_dir,
    f"DE_stats_template_data_{project_id}_real.txt"
)

# Metadata file with sample grouping to define comparison
metadata_filename = os.path.join(
    base_dir,
    dataset_name,
    "data",
    "metadata",
    f"{project_id}_groups.tsv"
)

## Enrichment methods
* [ROAST](https://pubmed.ncbi.nlm.nih.gov/20610611/) is available in limma
* [CAMERA](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3458527/) is available in limma
* [GSVA](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3618321/) its own bioconductor package
* [ORA]() is available in PathwayStudios or David

TO DO: Write about each method

In [6]:
# Create "<local_dir>/GSEA_stats/" subdirectory
os.makedirs(os.path.join(local_dir, "GSA_stats"), exist_ok=True)

In [7]:
# Load pathway data
hallmark_DB_filename = params["pathway_DB_filename"]

**Apply enrichment to template experiment**

See supplementary tables: https://academic.oup.com/bib/advance-article/doi/10.1093/bib/bbz158/5722384

In [8]:
%%R -i base_dir -i local_dir -i project_id -i template_expression_filename -i hallmark_DB_filename -i metadata_filename -i enrichment_method -o template_enriched_pathways

source(paste0(base_dir, '/generic_expression_patterns_modules/other_enrichment_methods.R'))

out_filename <- paste(local_dir, 
                      "GSA_stats/",
                      enrichment_method,
                      "_stats_template_data_",
                      project_id,
                      "_real.txt", 
                      sep = "")

if (enrichment_method == "GSVA"){
    
    template_enriched_pathways <- find_enriched_pathways_GSVA(template_expression_filename, hallmark_DB_filename)
}
else if (enrichment_method == "ROAST"){
    
    template_enriched_pathways <- find_enriched_pathways_ROAST(template_expression_filename, metadata_filename, hallmark_DB_filename)
}
else if (enrichment_method == "CAMERA"){
    
    template_enriched_pathways <- find_enriched_pathways_CAMERA(template_expression_filename, metadata_filename, hallmark_DB_filename)
}
write.table(as.data.frame(template_enriched_pathways), file = out_filename, row.names = F, sep = "\t")

/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loading required package: Rcpp

  warnings.warn(x, RRuntimeWarning)
/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 
Attaching package: ‘edgeR’


  warnings.warn(x, RRuntimeWarning)
/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: The following object is masked from ‘package:DEFormats’:

    DGEList


  warnings.warn(x, RRuntimeWarning)


[1] "Checking sample ordering..."
[1] TRUE


/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


In [9]:
# Quick check
print(template_enriched_pathways.shape)
template_enriched_pathways

(50, 4)


,NGenes,Direction,PValue,FDR
0,181.0,Down,2.186927e-13,1.093464e-11
1,183.0,Up,1.185204e-10,2.963010e-09
2,180.0,Down,6.409730e-10,1.068288e-08
3,138.0,Down,3.588686e-08,4.485858e-07
4,193.0,Up,9.504961e-08,9.504961e-07
5,197.0,Up,5.561857e-07,4.634881e-06
6,197.0,Up,3.024132e-06,2.160094e-05
7,192.0,Up,5.097628e-06,3.186017e-05
8,93.0,Down,8.104483e-06,4.405736e-05
9,180.0,Up,9.065422e-06,4.405736e-05


**Apply enrichment to simulated experiments**

In [10]:
## TO DO: Check issues
## EA stats not outputting in correct location for some reason for GSVA.
## All stats are the same using ROAST

In [11]:
%%R -i project_id -i local_dir -i hallmark_DB_filename -i metadata_filename -i num_runs -i base_dir -i enrichment_method

source(paste0(base_dir, '/generic_expression_patterns_modules/other_enrichment_methods.R'))

for (i in 0:(num_runs-1)){
    simulated_expression_filename <- paste(local_dir, 
                                           "pseudo_experiment/selected_simulated_data_",
                                           project_id,
                                           "_", 
                                           i,
                                           "_processed.txt",
                                           sep = "")

    out_filename <- paste(local_dir,
                          "GSA_stats/",
                          enrichment_method,
                          "_stats_simulated_data_",
                          project_id,
                          "_",
                          i,
                          ".txt", 
                          sep = "")
    
    if (enrichment_method == "GSVA"){
        enriched_pathways <- find_enriched_pathways_GSVA(simulated_expression_filename, hallmark_DB_filename) 
        write.table(as.data.frame(enriched_pathways), file = out_filename, row.names = F, sep = "\t")
        print("in GSVA")
    }
    else if (enrichment_method == "ROAST"){
        enriched_pathways <- find_enriched_pathways_ROAST(simulated_expression_filename, metadata_filename, hallmark_DB_filename) 
        write.table(as.data.frame(enriched_pathways), file = out_filename, row.names = F, sep = "\t")
        print("in ROAST")
    }
    else if (enrichment_method == "CAMERA"){
        enriched_pathways <- find_enriched_pathways_CAMERA(simulated_expression_filename, metadata_filename, hallmark_DB_filename) 
        write.table(as.data.frame(enriched_pathways), file = out_filename, row.names = F, sep = "\t")
        print("in CAMERA")
    }
}

[1] "Checking sample ordering..."
[1] TRUE
[1] "in CAMERA"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in CAMERA"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in CAMERA"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in CAMERA"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in CAMERA"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in CAMERA"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in CAMERA"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in CAMERA"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in CAMERA"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in CAMERA"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in CAMERA"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in CAMERA"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in CAMERA"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in CAMERA"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in CAMERA"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in CAMERA"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in CAMER

### TO DO:
Validate results. Looks like all pathways have the same statistic for ROAST

## Format enrichment output

Each method yields a different output format so we will need to format the data before we can rank and summarize it

In [12]:
%%R -i hallmark_DB_filename -o hallmark_DB_names
library("GSA")

hallmark_DB <- GSA.read.gmt(hallmark_DB_filename)

hallmark_DB_names <- as.data.frame(hallmark_DB$geneset.names)

/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Read 50 records

  warnings.warn(x, RRuntimeWarning)
/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Read 7424 items

  warnings.warn(x, RRuntimeWarning)


12345678910111213141516171819202122232425262728293031323334353637383940414243444546474849501
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [13]:
ranking.format_enrichment_output(
    local_dir, 
    project_id, 
    enrichment_method, 
    hallmark_DB_names,
    num_runs
)

## Rank pathways

In [14]:
analysis_type = "GSA"

template_GSEA_stats_filename = os.path.join(
    local_dir,
    "EA_stats",
    f"{enrichment_method}_stats_template_data_{project_id}_real.txt"    
)
template_GSEA_stats, simulated_GSEA_summary_stats = ranking.process_and_rank_genes_pathways(
    template_GSEA_stats_filename,
    local_dir,
    num_runs,
    project_id,
    analysis_type,
    col_to_rank_pathways,
    enrichment_method
)

                             Direction        PValue           FDR
pathway                                                           
HALLMARK_ADIPOGENESIS             Down  1.021544e-12  5.107719e-11
HALLMARK_ALLOGRAFT_REJECTION        Up  5.255070e-12  1.313767e-10
HALLMARK_ANDROGEN_RESPONSE        Down  4.751489e-11  7.919148e-10
HALLMARK_ANGIOGENESIS               Up  4.096241e-10  5.120302e-09
HALLMARK_APICAL_JUNCTION          Down  2.060612e-09  2.060612e-08
                             Direction        PValue           FDR
pathway                                                           
HALLMARK_ADIPOGENESIS               Up  1.741286e-09  8.706429e-08
HALLMARK_ALLOGRAFT_REJECTION      Down  2.616560e-08  6.541401e-07
HALLMARK_ANDROGEN_RESPONSE          Up  2.563545e-07  4.272574e-06
HALLMARK_ANGIOGENESIS               Up  1.726077e-06  2.157597e-05
HALLMARK_APICAL_JUNCTION            Up  2.581570e-06  2.581570e-05
                             Direction        PValue          

## Pathway summary table

In [18]:
# Create intermediate file: "<local_dir>/gene_summary_table_<col_to_rank_pathways>.tsv"
summary_pathway_ranks = ranking.generate_summary_table(
    template_GSEA_stats_filename,
    template_GSEA_stats,
    simulated_GSEA_summary_stats,
    col_to_rank_pathways,
    local_dir,
    'pathway',
    params
)

summary_pathway_ranks.sort_values(by="Z score", ascending=False).head()

                             Direction        PValue           FDR  ranking  \
HALLMARK_ADIPOGENESIS             Down  2.186927e-13  1.093464e-11     50.0   
HALLMARK_ALLOGRAFT_REJECTION        Up  1.185204e-10  2.963010e-09     49.0   
HALLMARK_ANDROGEN_RESPONSE        Down  6.409730e-10  1.068288e-08     48.0   
HALLMARK_ANGIOGENESIS             Down  3.588686e-08  4.485858e-07     47.0   
HALLMARK_APICAL_JUNCTION            Up  9.504961e-08  9.504961e-07     46.0   

                              (FDR, median)  (FDR, mean)  (FDR, std)  \
HALLMARK_ADIPOGENESIS          3.739628e-08     0.000040    0.000154   
HALLMARK_ALLOGRAFT_REJECTION   1.847834e-07     0.000100    0.000418   
HALLMARK_ANDROGEN_RESPONSE     4.272574e-06     0.000176    0.000467   
HALLMARK_ANGIOGENESIS          7.220993e-06     0.000379    0.000946   
HALLMARK_APICAL_JUNCTION       1.107009e-05     0.000931    0.002421   

                              (FDR, count)  (ranking, )  
HALLMARK_ADIPOGENESIS             

KeyError: 'the label [padj] is not in the [columns]'

In [ ]:
# Create `pathway_summary_filename`
summary_pathway_ranks.to_csv(pathway_summary_filename, sep='\t')